In [1]:
import pandas as pd
import json
import numpy as np
import seaborn as sn
import pickle

from matplotlib import pyplot as plt
from io import StringIO
%matplotlib inline

In [ ]:
import os
import ssl
import requests

from tqdm import tqdm
from PIL import Image
from io import BytesIO
from urllib import request

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_dir = '/content/drive/MyDrive/colab_nbs/airbnb/input/'
output_dir = '/content/drive/MyDrive/colab_nbs/airbnb/output/'
os.makedirs(output_dir, exist_ok=True)
print(f"Directory ensures existence: {output_dir}")
images_dir = '/content/drive/MyDrive/colab_nbs/airbnb/images'
os.makedirs(extracted_dir, exist_ok=True)
print(f"Directory ensures existence: {extracted_dir}")

Directory ensures existence: /content/drive/MyDrive/colab_nbs/airbnb/output/
Directory ensures existence: /content/drive/MyDrive/colab_nbs/airbnb/images


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
df_listings_details = pd.read_csv(input_dir + 'listings_detailed.csv')
df_listings = pd.read_csv(input_dir + 'listings.csv')

In [ ]:
selected_columns = ['id','room_type', 'minimum_nights', 'neighbourhood',
   'availability_eoy', 'availability_365', 'picture_url',
    'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_identity_verified',
    'accommodates', 'bathrooms', 'bedrooms', 'beds',
    'estimated_occupancy_l365d', 'estimated_revenue_l365d',
    'number_of_reviews', 'number_of_reviews_l30d', 'reviews_per_month',
    'review_scores_rating', 'review_scores_value',
    'instant_bookable', 'calculated_host_listings_count', 'price']


In [ ]:
def normalize_tf_cols(df, column):
    df[column] = df[column].replace({'t': 1, 'f': 0}).astype(bool)
    return df

In [ ]:
def fix_encoding(df_cleaned):
    encoding_map = {}
    for val in df_cleaned['neighbourhood'].unique():
        try:
            clean_val = val.encode("latin1").decode("utf-8", errors="ignore")
            encoding_map[val] = clean_val
        except (UnicodeEncodeError, AttributeError):
            encoding_map[val] = val
    df_cleaned['neighbourhood'] = df_cleaned['neighbourhood'].map(encoding_map)
    return df_cleaned

In [ ]:
def data_cleanup(df_1, df_2):
    df_merged_listings = pd.concat([df_listings, df_listings_details], axis=1)
    df_merged_listings = df_merged_listings.loc[:, ~df_merged_listings.columns.duplicated()]
    df_cleaned = df_merged_listings[selected_columns].dropna()
    df_cleaned = df_cleaned[df_cleaned['availability_eoy']> 0]
    df_cleaned = df_cleaned[df_cleaned['availability_365']> 0]
    df_cleaned = df_cleaned[df_cleaned['estimated_occupancy_l365d']> 0]
    df_cleaned['host_response_rate'] = df_cleaned['host_response_rate'].str.replace('%', '', regex=False).astype(float)
    df_cleaned['host_acceptance_rate'] = df_cleaned['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
    df_cleaned = normalize_tf_cols(df_cleaned, 'instant_bookable')
    df_cleaned = normalize_tf_cols(df_cleaned, 'host_identity_verified')
    df_cleaned = normalize_tf_cols(df_cleaned, 'host_is_superhost')
    df_cleaned = fix_encoding(df_cleaned)
    df_cleaned.columns = df_cleaned.columns.str.replace('/','_')
    df_cleaned.columns = df_cleaned.columns.str.lower()
    df_cleaned.columns = df_cleaned.columns.str.replace(' ','_')
    return df_cleaned

In [ ]:
df_cleaned = data_cleanup(df_listings, df_listings_details)
df_cleaned = df_cleaned.reset_index(drop=True)

In [ ]:
def identify_premium_properties(df, threshold=0.5):
    neighborhood_premium_stats = df_cleaned.groupby('neighbourhood').agg({
        'id':'count',
        'price':  lambda x: x.quantile(threshold),
        'review_scores_value': lambda x: x.quantile(threshold)
    })
    neighborhood_premium_stats = neighborhood_premium_stats.rename(
    columns={
        'price': 'price_q_threshold',
        'review_scores_value': 'rating_q_threshold'
    })
    neighborhood_premium_stats = neighborhood_premium_stats.reset_index()
    df_premium = df_cleaned.merge(
        neighborhood_premium_stats[['neighbourhood', 'price_q_threshold', 'rating_q_threshold']],
        on='neighbourhood',
        how='left'
    )
    df_premium['is_premium'] = (
            (df_premium['price'] >= df_premium['price_q_threshold']) &
            (df_premium['review_scores_value'] >= df_premium['rating_q_threshold'])
        )
    df_premium['is_premium'] = df_premium['is_premium'].astype(int)
    return df_premium

In [ ]:
df_premium = identify_premium_properties(df_cleaned, 0.5)

In [ ]:
df_premium[['id', 'picture_url']].head(2)

,id,picture_url
0,51287,https://a0.muscache.com/pictures/25163038/1c4e...
1,169672,https://a0.muscache.com/pictures/c1a1e093-66da...


In [ ]:
def download_image_from_url(url):
    context = ssl._create_unverified_context()
    try:
        with request.urlopen(url, context=context) as resp:
            buffer = resp.read()
        stream = BytesIO(buffer)
        img = Image.open(stream)
        return img
    except:
        print("Image Not found exception")
        return None
    print("Image Not found")
    return None

In [ ]:
def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [ ]:
sample_df = df_premium.sample(100, random_state=42)

for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    url = row['picture_url']
    listing_id = row['id']

    if pd.isna(url):
        continue

    if row['is_premium'] == 1:
        folder = images_dir + "/premium"
    else:
        folder = images_dir + "/non_premium"

    save_path = f"{folder}/{listing_id}.jpg"
    print(save_path)
    img = download_image_from_url(url)

    if img is None:
        print(img)
        continue;

    resized_img =  prepare_image(img, (300,400))
    resized_img.save(save_path)

  0%|          | 0/100 [00:00<?, ?it/s]

/content/drive/MyDrive/colab_nbs/airbnb/images/non_premium/964206565905955937.jpg


  1%|          | 1/100 [00:01<01:41,  1.03s/it]

/content/drive/MyDrive/colab_nbs/airbnb/images/non_premium/1102298634083944389.jpg


  2%|▏         | 2/100 [00:01<01:25,  1.14it/s]

...

 98%|█████████▊| 98/100 [00:44<00:00,  2.45it/s]

/content/drive/MyDrive/colab_nbs/airbnb/images/non_premium/44027879.jpg


 99%|█████████▉| 99/100 [00:44<00:00,  2.78it/s]

/content/drive/MyDrive/colab_nbs/airbnb/images/non_premium/34876910.jpg


100%|██████████| 100/100 [00:45<00:00,  2.20it/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

img_size = (224, 224)
batch_size = 10

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    images_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode="binary"
)

Found 99 files belonging to 2 classes.


In [ ]:
train_ds = dataset.take(int(len(dataset)*0.8))
val_ds   = dataset.skip(int(len(dataset)*0.8))
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(tf.data.AUTOTUNE)
len(dataset), len(train_ds), len(val_ds)

(10, 8, 2)

In [ ]:
augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1)
])

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import models

base = EfficientNetB0(
    include_top=False,
    input_shape=img_size + (3,),
    weights="imagenet"
)

base.trainable = False


In [ ]:
inputs = layers.Input(shape=img_size + (3,))
x = augment(inputs)
x = tf.keras.applications.efficientnet.preprocess_input(x)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.4)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=[0]* 75 + [1]* 24
)

class_weights = {0: class_weights[0], 1: class_weights[1]}
class_weights

{0: np.float64(0.66), 1: np.float64(2.0625)}

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=8,
    class_weight=class_weights
)

Epoch 1/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.4495 - loss: 0.7559 - val_accuracy: 0.8421 - val_loss: 0.5207
Epoch 2/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 977ms/step - accuracy: 0.6526 - loss: 0.6495 - val_accuracy: 0.6316 - val_loss: 0.6433
Epoch 3/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 802ms/step - accuracy: 0.6140 - loss: 0.5580 - val_accuracy: 0.8947 - val_loss: 0.4128
Epoch 4/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.9210 - loss: 0.4426 - val_accuracy: 0.8947 - val_loss: 0.3795
Epoch 5/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 809ms/step - accuracy: 0.8205 - loss: 0.3468 - val_accuracy: 0.7368 - val_loss: 0.5299
Epoch 6/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8171 - loss: 0.4389 - val_accuracy: 0.8947 - val_loss: 0.2476
Epoch 7/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 972ms/step - accuracy: 0.9358 - loss: 0.3071 - val_accuracy: 0.9474 - val_loss: 0.2494
Epoch 8/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 808ms/step - accuracy: 0.8325 - loss: 0.3538 - val_accuracy: 1.0000 - val_loss: 0.1476


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true = np.concatenate([y for _, y in val_ds], axis=0)
y_pred = model.predict(val_ds).ravel()
y_class = (y_pred >= 0.46).astype(int)
print(confusion_matrix(y_true, y_class))
print(classification_report(y_true, y_class))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 438ms/step
[[11  3]
 [ 1  4]]
              precision    recall  f1-score   support

         0.0       0.92      0.79      0.85        14
         1.0       0.57      0.80      0.67         5

    accuracy                           0.79        19
   macro avg       0.74      0.79      0.76        19
weighted avg       0.83      0.79      0.80        19



In [ ]:
model.save(output_dir + "cnn_premium_detector.keras")

In [ ]:
sample_df = df_premium.sample(100, random_state=30)

for _, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    url = row['picture_url']
    listing_id = row['id']

    if pd.isna(url):
        continue

    if row['is_premium'] == 1:
        folder = images_dir + "/test/premium"
    else:
        folder = images_dir + "/test/non_premium"

    save_path = f"{folder}/{listing_id}.jpg"
    print(save_path)
    img = download_image_from_url(url)

    if img is None:
        print(img)
        continue;

    resized_img =  prepare_image(img, (300,400))
    resized_img.save(save_path)

  0%|          | 0/100 [00:00<?, ?it/s]

/content/drive/MyDrive/colab_nbs/airbnb/images/test/non_premium/948991807741346465.jpg


  1%|          | 1/100 [00:00<00:31,  3.13it/s]

/content/drive/MyDrive/colab_nbs/airbnb/images/test/non_premium/14726391.jpg


  2%|▏         | 2/100 [00:00<00:33,  2.94it/s]

...

/content/drive/MyDrive/colab_nbs/airbnb/images/test/non_premium/8049046.jpg
/content/drive/MyDrive/colab_nbs/airbnb/images/test/non_premium/1218675917828964223.jpg


 98%|█████████▊| 98/100 [00:38<00:00,  3.29it/s]

/content/drive/MyDrive/colab_nbs/airbnb/images/test/non_premium/978051288949733487.jpg


100%|██████████| 100/100 [00:38<00:00,  2.59it/s]

/content/drive/MyDrive/colab_nbs/airbnb/images/test/non_premium/43976150.jpg


In [ ]:
def predict_image(path):
    img = Image.open(path).convert("RGB")
    img = img.resize(img_size)

    arr = np.array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)

    prob = model.predict(arr)[0][0]

    print("Path:", path)
    print("Premium probability:", prob)

    if prob >= 0.6:
        print("Prediction → PREMIUM")
        return True
    else:
        print("Prediction → NON-PREMIUM")
        return False

In [ ]:
test_dir = '/content/drive/MyDrive/colab_nbs/airbnb/test'
test_folder = test_dir + "/images"
non_premium_count = 0
premium_count = 0
for filename in os.listdir(test_folder):
  full_path = os.path.join(test_folder, filename)
  result = predict_image(full_path)
  if(result is True):
    premium_count = premium_count + 1
  else:
    non_premium_count = non_premium_count + 1
print("Total Premium", premium_count )
print("Total Non-Premium", non_premium_count )

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Path: /content/drive/MyDrive/colab_nbs/airbnb/test/images/29901052.jpg
Premium probability: 0.7225854
Prediction → PREMIUM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Path: /content/drive/MyDrive/colab_nbs/airbnb/test/images/302499.jpg
Premium probability: 0.7228488
Prediction → PREMIUM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Path: /content/drive/MyDrive/colab_nbs/airbnb/test/images/2750261.jpg
Premium probability: 0.72347444
Prediction → PREMIUM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Path: /content/drive/MyDrive/colab_nbs/airbnb/test/images/45853532.jpg
Premium probability: 0.7191277
Prediction → PREMIUM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Path: /content/drive/MyDrive/colab_nbs/airbnb/test/images/44027854.jpg
Premium probability: 0.72373205
Prediction → PREMIUM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Path: /content/drive/MyDrive/colab_nbs/airbnb/test/images/5835467.jpg
Premium probability: 0.72255766
Prediction → PREMIUM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/s